In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { min-width:1500px !important; }</style>"))

import os
import sys
sys.path.append('..')

import time
import datetime

import importlib

import tensorflow as tf
from tensorflow import keras

import src
from src.data import cifar100
from src.models import basic, factory
from src.meta.meta import Metadata
from src.utils import debug_utils, file_utils, io_utils

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S   '), tf.__name__, '-', tf.__version__, sep='')

W0813 16:57:49.135205 140036826816640 __init__.py:689] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


2019-08-13 16:57:49   tensorflow-1.15.0-dev20190813


In [2]:
from src.utils.logger import Logging
logger = Logging.get_logger(__name__)
Logging.attach_stdout()

# Setup

In [3]:
_NAME = basic.NAME
_EPOCH = 10

filepath_md = file_utils.model_filepath_md(_NAME, _EPOCH)
print('filepath_md:            ', filepath_md)
filepath_pb = file_utils.model_filepath_pb(_NAME, _EPOCH)
print('filepath_pb:            ', filepath_pb)
filepath_weights_h5 = file_utils.model_filepath_weights_h5(_NAME, _EPOCH)
print('filepath_weights_h5:    ', filepath_weights_h5)
filepath_tflite = file_utils.model_filepath_tflite(_NAME, _EPOCH)
print('filepath_tflite:        ', filepath_tflite)
filepath_edgetpu_tflite = file_utils.model_filepath_edgetpu_tflite(_NAME, _EPOCH)
print('filepath_edgetpu_tflite:', filepath_edgetpu_tflite)

output_data_file = 'tmp/test_data_%s.csv' % datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
print('output_data_file:       ', output_data_file)

TRIALS = 10
TEST_SIZES = [1, 10, 100, 1000]
REPEAT = 2

filepath_md:             /space/code/deep-learning/models/basic/010/basic_010.md
filepath_pb:             /space/code/deep-learning/models/basic/010/basic_010.pb
filepath_weights_h5:     /space/code/deep-learning/models/basic/010/basic_010_weights.h5
filepath_tflite:         /space/code/deep-learning/models/basic/010/basic_010_int8.tflite
filepath_edgetpu_tflite: /space/code/deep-learning/models/basic/010/basic_010_int8_edgetpu.tflite
output_data_file:        tmp/test_data_2019-08-13-16:57:49.csv


In [4]:
%%bash

find ../models | grep basic | grep -E '(010|009)' | sort

../models/basic/009
../models/basic/009/basic_009_checkpoint.h5
../models/basic/009/basic_009.h5
../models/basic/009/basic_009.md
../models/basic/009/basic_009_weights.h5
../models/basic/010
../models/basic/010/basic_010_checkpoint.h5
../models/basic/010/basic_010.h5
../models/basic/010/basic_010_int8_edgetpu.log
../models/basic/010/basic_010_int8_edgetpu.tflite
../models/basic/010/basic_010_int8.tflite
../models/basic/010/basic_010.md
../models/basic/010/basic_010.pb
../models/basic/010/basic_010_weights.h5


# Data

In [5]:
from src.data import cifar100

In [6]:
(TRAIN_X, TRAIN_Y), (TEST_X, TEST_Y) = cifar100.load_data(normalize=True)

# Inference Timing Funcs

In [7]:
class Row:
    def __init__(self, tag, trials, repeat_index, input_size, total_time, mean_time):
        self.tag = tag
        self.trials = trials
        self.repeat_index = repeat_index
        self.input_size = input_size
        self.total_time = total_time
        self.mean_time = mean_time
        
    def to_header(self):
        return ','.join(['model', 'dataset', 'trials', 'tag', 'input_size', 'repeat_index', 'total_time', 'mean_time'])
    
    def to_csv(self):
        return ','.join([str(s) for s in [_NAME, cifar100.NAME, self.trials, self.tag, self.input_size, self.repeat_index, self.total_time, self.mean_time]])

    
    
def write_data_row(row):
    if not os.path.exists(output_data_file):
        with open(output_data_file, 'w') as f:
            f.write(row.to_header() + '\n')
        
    with open(output_data_file, 'a') as f:
        f.write(row.to_csv() + '\n')
    


def predict(tag, model, test_images, trials, repeat_index, log_first=0, log_last=0, emit=False):
    start = time.time()
    longest_line = 0
    timings = []
    for i in range(0, trials):
        now = time.time()
        ret, y = model.predict(test_images)
        if ret != 0:
            logger.error('Failed predicting with error %d', ret)
            return ret, None
        elapsed = 1000 * (time.time() - now)
        running_avg = (time.time() - start) / (i + 1)

        trials_format = '%d'
        log = (trials_format + ': %.3fms') % (i + 1, elapsed)
        if i < log_first or i > trials - log_last:
            print(log)
        else:
            if i + 1 != trials:
                log += ' ['
                log += '=' * i
                log += '>'
                log += '.' * (trials - i - 1)
                log += ']'
                remaining_time = running_avg * (trials - i - 1)
                log += ' ETA: %.3fs' % remaining_time
            longest_line = max(len(log), longest_line)
            if len(log) < longest_line:
                log += ' ' * (longest_line - len(log))
            print('\r%s' % log, end='\r')
        
        timings.append(elapsed)
    
    row = Row(tag=tag, trials=trials, repeat_index=repeat_index, input_size=len(test_images), total_time=sum(timings), mean_time=sum(timings)/len(timings))
    if emit:
        write_data_row(row)
    return 0, row

def warm_up(tag, model, test_images, trials, repeat):
    assert tag is not None and isinstance(tag, str)
    logger.info('%s Warming up on %d images for %d iterations...' % (model.name, len(test_images), trials))
    timings = []
    for i in range(repeat):
        ret, row = predict(tag, model, test_images, trials, i + 1)
        if ret != 0:
            return ret
        log_images_size = ('%d' % row.input_size).ljust(5)
        log_prefix_ = ('[%03dx%s %s]') % (row.trials, log_images_size, tag)
        logger.info('%s Warm up = %.5fs (avg = %.3fms)' % (log_prefix_, row.total_time / 1000, row.mean_time))

    return 0


def test(tag, model, test_images, trials, repeat):
    assert tag is not None and isinstance(tag, str)
    if len(test_images) == 1:
        phrase = 'single image'
    else:
        phrase = 'batch of %d images' % len(test_images)
    logger.info('%s Repeat %dx timing on %s for %d iterations...', model.name, repeat, phrase, trials)
    for i in range(repeat):
        ret, row = predict(tag, model, test_images, trials, i + 1, emit=True)
        if ret != 0:
            return ret
        log_images_size = ('%d' % row.input_size).ljust(5)
        log_prefix_ = ('[%03dx%s %s]') % (row.trials, log_images_size, tag)
        logger.info('%s Test =    %.5fs (avg = %.3fms)' % (log_prefix_, row.total_time / 1000, row.mean_time))

    return 0

# Edge TPU

In [8]:
from src.tflite.model import TfLiteModel
importlib.reload(src.tflite.model)
from src.tflite.model import TfLiteModel

In [9]:
ret, tpu_model = TfLiteModel.from_tflite(filepath_md, filepath_edgetpu_tflite)
assert ret == 0
tpu_model.dump()

2019-08-13 16:57:49,996 DEBUG [src.tflite.model:75] (MainThread) Loading metadata from /space/code/deep-learning/models/basic/010/basic_010.md...
2019-08-13 16:57:49,998 DEBUG [src.tflite.model:80] (MainThread) basic Loading tflite interpreter from /space/code/deep-learning/models/basic/010/basic_010_int8_edgetpu.tflite...
2019-08-13 16:57:50,000 INFO  [src.tflite.model:86] (MainThread) Using experimental library libedgetpu.so.1.0...
2019-08-13 16:57:52,706 DEBUG [src.tflite.model:57] (MainThread) Input mean, std, index: 0, 0.003921568859368563, 0
2019-08-13 16:57:52,706 DEBUG [src.tflite.model:62] (MainThread) Output mean, std, index: 0, 0.00390625, 1


0

In [10]:
ret, y_tflite = tpu_model.predict(TEST_X)
assert ret == 0

In [11]:
for i in TEST_SIZES:
    warm_up('edgetpu', tpu_model, TEST_X[:i], TRIALS, 1)
    test('edgetpu', tpu_model, TEST_X[:i], TRIALS, REPEAT)

2019-08-13 16:57:55,183 INFO  [__main__:68] (MainThread) basic Warming up on 1 images for 10 iterations...
2019-08-13 16:57:55,188 INFO  [__main__:76] (MainThread) [010x1     edgetpu] Warm up = 0.00436s (avg = 0.436ms)
2019-08-13 16:57:55,189 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on single image for 10 iterations...
2019-08-13 16:57:55,195 INFO  [__main__:94] (MainThread) [010x1     edgetpu] Test =    0.00380s (avg = 0.380ms)
2019-08-13 16:57:55,199 INFO  [__main__:94] (MainThread) [010x1     edgetpu] Test =    0.00341s (avg = 0.341ms)
2019-08-13 16:57:55,200 INFO  [__main__:68] (MainThread) basic Warming up on 10 images for 10 iterations...
2019-08-13 16:57:55,226 INFO  [__main__:76] (MainThread) [010x10    edgetpu] Warm up = 0.02478s (avg = 2.478ms)
2019-08-13 16:57:55,227 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on batch of 10 images for 10 iterations...
2019-08-13 16:57:55,254 INFO  [__main__:94] (MainThread) [010x10    edgetpu] Test =    0.02647s (

In [12]:
del tpu_model

# Keras

In [13]:
from src.keras.model import KerasModel
importlib.reload(src.keras.model)
from src.keras.model import KerasModel

In [14]:
f_construct_keras_model = factory.get_model_create_func(_NAME, cifar100.INPUT_SHAPE, len(cifar100.CLASSES))
ret, keras_model = KerasModel.from_weights_h5(filepath_md, f_construct_keras_model, filepath_weights_h5)
assert ret == 0

2019-08-13 16:58:03,419 DEBUG [src.keras.model:90] (MainThread) [basic|10] Constructing keras model from factory function...


W0813 16:58:03.440439 140036826816640 deprecation.py:506] From /home/maka/.pyenv/versions/3.6.8/envs/deep-learning_nightly/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1633: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2019-08-13 16:58:03,449 DEBUG [src.keras.model:95] (MainThread) [basic|10] Loading keras model weights from /space/code/deep-learning/models/basic/010/basic_010_weights.h5...
2019-08-13 16:58:04,027 DEBUG [src.keras.model:39] (MainThread) [basic|10|keras] Setting metadata input names to ['basic0:0']
2019-08-13 16:58:04,028 DEBUG [src.keras.model:47] (MainThread) [basic|10|keras] Setting metadata out names to ['basic2/Softmax:0']


In [15]:
ret, y_keras = keras_model.predict(TEST_X)
assert ret == 0
keras_model.dump()

2019-08-13 16:58:04,513 DEBUG [src.meta.metadata:62] (MainThread) [basic|10|keras] epoch = 10
2019-08-13 16:58:04,514 DEBUG [src.meta.metadata:63] (MainThread) [basic|10|keras] inputs = ['basic0:0']
2019-08-13 16:58:04,515 DEBUG [src.meta.metadata:64] (MainThread) [basic|10|keras] outputs = ['basic2/Softmax:0']


In [16]:
for i in TEST_SIZES:
    test('keras', keras_model, TEST_X[:i], TRIALS, REPEAT)

2019-08-13 16:58:04,521 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on single image for 10 iterations...
2019-08-13 16:58:04,538 INFO  [__main__:94] (MainThread) [010x1     keras] Test =    0.01515s (avg = 1.515ms)
2019-08-13 16:58:04,555 INFO  [__main__:94] (MainThread) [010x1     keras] Test =    0.01410s (avg = 1.410ms)
2019-08-13 16:58:04,555 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on batch of 10 images for 10 iterations...
2019-08-13 16:58:04,574 INFO  [__main__:94] (MainThread) [010x10    keras] Test =    0.01740s (avg = 1.740ms)
2019-08-13 16:58:04,593 INFO  [__main__:94] (MainThread) [010x10    keras] Test =    0.01670s (avg = 1.670ms)
2019-08-13 16:58:04,593 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on batch of 100 images for 10 iterations...
2019-08-13 16:58:04,649 INFO  [__main__:94] (MainThread) [010x100   keras] Test =    0.05347s (avg = 5.347ms)
2019-08-13 16:58:04,700 INFO  [__main__:94] (MainThread) [010x100   keras] Test =    0

In [17]:
del keras_model

2019-08-13 16:58:05,385 INFO  [src.keras.model:63] (MainThread) basic Closing session...


# tensorflow

In [18]:
from src.tf.model import TensorFlowModel
importlib.reload(src.tf.model)
from src.tf.model import TensorFlowModel

In [19]:
ret, tf_model = TensorFlowModel.from_pb(filepath_md, filepath_pb)
assert ret == 0

2019-08-13 16:58:05,399 DEBUG [src.tf.model:96] (MainThread) Loading metadata from /space/code/deep-learning/models/basic/010/basic_010.md...
2019-08-13 16:58:05,400 DEBUG [src.tf.model:101] (MainThread) basic Loading frozen graph model from /space/code/deep-learning/models/basic/010/basic_010.pb...


W0813 16:58:05.401313 140036826816640 module_wrapper.py:137] From ../src/tf/model.py:102: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0813 16:58:05.404999 140036826816640 module_wrapper.py:137] From ../src/tf/model.py:106: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0813 16:58:05.420995 140036826816640 module_wrapper.py:137] From ../src/tf/model.py:114: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



2019-08-13 16:58:05,430 DEBUG [src.tf.model:118] (MainThread) basic pb model imported into tensorboard. Visualize by running: 
2019-08-13 16:58:05,431 DEBUG [src.tf.model:119] (MainThread) > tensorboard --logdir=/space/code/deep-learning/tmp/logs/tensorboard/basic/20190813-165805
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:0b:00.0, compute capability: 7.5



In [20]:
ret, y_tf = tf_model.predict(TEST_X)
assert ret == 0
tf_model.dump()

2019-08-13 16:58:05,610 DEBUG [src.tf.model:82] (MainThread) basic graph_def.node names: ['basic0', 'basic1/Shape', 'basic1/strided_slice/stack', 'basic1/strided_slice/stack_1', 'basic1/strided_slice/stack_2', 'basic1/strided_slice', 'basic1/Reshape/shape/1', 'basic1/Reshape/shape', 'basic1/Reshape', 'basic2/kernel', 'basic2/bias', 'basic2/MatMul/ReadVariableOp', 'basic2/MatMul', 'basic2/BiasAdd/ReadVariableOp', 'basic2/BiasAdd', 'basic2/Softmax', 'Adam/iter', 'Adam/beta_1', 'Adam/beta_2', 'Adam/decay', 'Adam/learning_rate', 'Adam/basic2/kernel/m', 'Adam/basic2/bias/m', 'Adam/basic2/kernel/v', 'Adam/basic2/bias/v']
2019-08-13 16:58:05,611 DEBUG [src.tf.model:85] (MainThread) basic graph.get_operations(): [25] ['basic/basic0', 'basic/basic1/Shape', 'basic/basic1/strided_slice/stack', 'basic/basic1/strided_slice/stack_1', 'basic/basic1/strided_slice/stack_2', 'basic/basic1/strided_slice', 'basic/basic1/Reshape/shape/1', 'basic/basic1/Reshape/shape', 'basic/basic1/Reshape', 'basic/basic2/

0

In [21]:
for i in TEST_SIZES:
    test('tensorflow', tf_model, TEST_X[:i], TRIALS, REPEAT)

2019-08-13 16:58:05,619 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on single image for 10 iterations...
2019-08-13 16:58:05,628 INFO  [__main__:94] (MainThread) [010x1     tensorflow] Test =    0.00775s (avg = 0.775ms)
2019-08-13 16:58:05,636 INFO  [__main__:94] (MainThread) [010x1     tensorflow] Test =    0.00694s (avg = 0.694ms)
2019-08-13 16:58:05,637 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on batch of 10 images for 10 iterations...
2019-08-13 16:58:05,648 INFO  [__main__:94] (MainThread) [010x10    tensorflow] Test =    0.00973s (avg = 0.973ms)
2019-08-13 16:58:05,658 INFO  [__main__:94] (MainThread) [010x10    tensorflow] Test =    0.00862s (avg = 0.862ms)
2019-08-13 16:58:05,659 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on batch of 100 images for 10 iterations...
2019-08-13 16:58:05,682 INFO  [__main__:94] (MainThread) [010x100   tensorflow] Test =    0.02087s (avg = 2.087ms)
2019-08-13 16:58:05,706 INFO  [__main__:94] (MainThread) [010

In [22]:
del tf_model

2019-08-13 16:58:05,941 INFO  [src.tf.model:53] (MainThread) basic Closing session...


# tf.lite

In [23]:
from src.tflite.model import TfLiteModel
importlib.reload(src.tflite.model)
from src.tflite.model import TfLiteModel

In [24]:
ret, tflite_model = TfLiteModel.from_tflite(filepath_md, filepath_tflite)
assert ret == 0
tflite_model.dump()

2019-08-13 16:58:05,955 DEBUG [src.tflite.model:75] (MainThread) Loading metadata from /space/code/deep-learning/models/basic/010/basic_010.md...
2019-08-13 16:58:05,956 DEBUG [src.tflite.model:80] (MainThread) basic Loading tflite interpreter from /space/code/deep-learning/models/basic/010/basic_010_int8.tflite...
2019-08-13 16:58:05,957 DEBUG [src.tflite.model:57] (MainThread) Input mean, std, index: 0, 0.003921568859368563, 5
2019-08-13 16:58:05,958 DEBUG [src.tflite.model:62] (MainThread) Output mean, std, index: 0, 0.00390625, 6


0

In [25]:
ret, y_tflite = tflite_model.predict(TEST_X)
assert ret == 0

In [26]:
for i in TEST_SIZES:
    test('tflite', tflite_model, TEST_X[:i], TRIALS, REPEAT)

2019-08-13 16:58:07,060 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on single image for 10 iterations...
2019-08-13 16:58:07,065 INFO  [__main__:94] (MainThread) [010x1     tflite] Test =    0.00272s (avg = 0.272ms)
2019-08-13 16:58:07,069 INFO  [__main__:94] (MainThread) [010x1     tflite] Test =    0.00313s (avg = 0.313ms)
2019-08-13 16:58:07,070 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on batch of 10 images for 10 iterations...
2019-08-13 16:58:07,087 INFO  [__main__:94] (MainThread) [010x10    tflite] Test =    0.01625s (avg = 1.625ms)
2019-08-13 16:58:07,100 INFO  [__main__:94] (MainThread) [010x10    tflite] Test =    0.01231s (avg = 1.231ms)
2019-08-13 16:58:07,101 INFO  [__main__:87] (MainThread) basic Repeat 2x timing on batch of 100 images for 10 iterations...
2019-08-13 16:58:07,205 INFO  [__main__:94] (MainThread) [010x100   tflite] Test =    0.10240s (avg = 10.240ms)
2019-08-13 16:58:07,316 INFO  [__main__:94] (MainThread) [010x100   tflite] Test

In [27]:
del tflite_model

# Results

In [35]:
%%bash

find tmp -type f | sort

tmp/test_data_2019-08-13-16:26:01.csv
tmp/test_data_2019-08-13-16:30:16.csv
tmp/test_data_2019-08-13-16:36:19.csv
tmp/test_data_2019-08-13-16:46:03.csv
tmp/test_data_2019-08-13-16:57:00.csv
tmp/test_data_2019-08-13-16:57:49.csv


In [36]:
%%bash
find tmp -type f | sort | tail -1 | xargs cat

model,dataset,trials,tag,input_size,repeat_index,total_time,mean_time
basic,cifar100,10,edgetpu,1,1,3.8008689880371094,0.38008689880371094
basic,cifar100,10,edgetpu,1,2,3.408193588256836,0.3408193588256836
basic,cifar100,10,edgetpu,10,1,26.46613121032715,2.646613121032715
basic,cifar100,10,edgetpu,10,2,26.03602409362793,2.603602409362793
basic,cifar100,10,edgetpu,100,1,234.06457901000977,23.406457901000977
basic,cifar100,10,edgetpu,100,2,240.79036712646484,24.079036712646484
basic,cifar100,10,edgetpu,1000,1,2411.196708679199,241.11967086791992
basic,cifar100,10,edgetpu,1000,2,2539.804697036743,253.98046970367432
basic,cifar100,10,keras,1,1,15.152692794799805,1.5152692794799805
basic,cifar100,10,keras,1,2,14.095067977905273,1.4095067977905273
basic,cifar100,10,keras,10,1,17.39811897277832,1.739811897277832
basic,cifar100,10,keras,10,2,16.704559326171875,1.6704559326171875
basic,cifar100,10,keras,100,1,53.47180366516113,5.347180366516113
basic,cifar100,10,keras,100,2,49.1485595703125,4.9

# Debug

In [29]:
from src.tf import utils
importlib.reload(src.tf.utils)

<module 'src.tf.utils' from '../src/tf/utils.py'>

In [30]:
help(utils)

Help on module src.tf.utils in src.tf:

NAME
    src.tf.utils

FUNCTIONS
    devices()
    
    get_available_gpus()
    
    gpu_device_name()
    
    is_gpu_available()

FILE
    /space/code/deep-learning/src/tf/utils.py




In [31]:
utils.devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7577985376503766366, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2387845424754494257
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6535275900652677817
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10691896935
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14143381435286180763
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:0b:00.0, compute capability: 7.5"]

In [32]:
utils.get_available_gpus()

['/device:GPU:0']

In [33]:
utils.is_gpu_available()


True

In [34]:
utils.gpu_device_name()


'/device:GPU:0'